In [2]:
import sys
sys.path.append('../')

from map_speecht5_hf import Mapping
import torch
from transformers import SpeechT5ForSpeechToText, SpeechT5ForTextToSpeech

import os
import pickle

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
model_asr = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr").to(device)
model_tts = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

In [5]:
ckpt = torch.load("../checkpoints/speecht5_base.pt", map_location=device)

In [25]:
for a,b in ckpt['model'].items():
    print(a, b.shape)

encoder.version torch.Size([1])
encoder.layers.0.self_attn.k_proj.weight torch.Size([768, 768])
encoder.layers.0.self_attn.k_proj.bias torch.Size([768])
encoder.layers.0.self_attn.v_proj.weight torch.Size([768, 768])
encoder.layers.0.self_attn.v_proj.bias torch.Size([768])
encoder.layers.0.self_attn.q_proj.weight torch.Size([768, 768])
encoder.layers.0.self_attn.q_proj.bias torch.Size([768])
encoder.layers.0.self_attn.out_proj.weight torch.Size([768, 768])
encoder.layers.0.self_attn.out_proj.bias torch.Size([768])
encoder.layers.0.self_attn_layer_norm.weight torch.Size([768])
encoder.layers.0.self_attn_layer_norm.bias torch.Size([768])
encoder.layers.0.fc1.weight torch.Size([3072, 768])
encoder.layers.0.fc1.bias torch.Size([3072])
encoder.layers.0.fc2.weight torch.Size([768, 3072])
encoder.layers.0.fc2.bias torch.Size([768])
encoder.layers.0.final_layer_norm.weight torch.Size([768])
encoder.layers.0.final_layer_norm.bias torch.Size([768])
encoder.layers.0.norm_k.weight torch.Size([64])

In [6]:
mapping_class = Mapping(model_asr, ckpt)

## Encoder

In [7]:
encoder_state_dict = mapping_class.encoder_state_dict

In [8]:
model_asr.speecht5.encoder.wrapped_encoder.load_state_dict(encoder_state_dict)

<All keys matched successfully>

In [9]:
mapping_folder = "../extracted/speecht5/mapping/"
if not os.path.exists(mapping_folder):
    os.makedirs(embeddings_folder)

with open(os.path.join(mapping_folder, 'encoder_state_dict.pickle'), 'wb') as handle:
    pickle.dump(encoder_state_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Speech Pre-net

In [10]:
speech_prenet_state_dict = mapping_class.speech_prenet_state_dict

In [11]:
model_asr.speecht5.encoder.prenet.load_state_dict(speech_prenet_state_dict)

<All keys matched successfully>

In [12]:
mapping_folder = "../extracted/speecht5/mapping/"
if not os.path.exists(mapping_folder):
    os.makedirs(embeddings_folder)

with open(os.path.join(mapping_folder, 'speech_prenet_state_dict.pickle'), 'wb') as handle:
    pickle.dump(speech_prenet_state_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Text Pre-net

In [13]:
text_prenet_state_dict = mapping_class.text_prenet_state_dict

In [15]:
text_prenet_state_dict

{'embed_tokens.weight': tensor([[ 0.2236,  0.1462,  0.2703,  ..., -0.5044, -0.3403, -0.2323],
         [-0.0267, -0.0211,  0.0806,  ...,  0.0826, -0.0250,  0.0142],
         [ 0.1836,  0.1970,  0.1066,  ..., -0.0246,  0.0242, -0.1271],
         ...,
         [-0.0473, -0.0490,  0.0584,  ...,  0.1119, -0.1022, -0.0686],
         [-0.0393, -0.1295,  0.0012,  ...,  0.0482, -0.0222, -0.3157],
         [-0.0368, -0.0320,  0.0710,  ...,  0.1121, -0.0775, -0.0605]],
        device='cuda:0'),
 'encode_positions.alpha': tensor(0.1270, device='cuda:0')}

In [19]:
for a,b in model_tts.speecht5.encoder.prenet.named_parameters():
    print(a)

embed_tokens.weight
encode_positions.alpha


In [28]:
model_tts.speecht5.encoder.prenet.encode_positions

SpeechT5ScaledPositionalEncoding(
  (dropout): Dropout(p=0.1, inplace=False)
)

In [14]:
model_tts.speecht5.encoder.prenet.load_state_dict(text_prenet_state_dict)

RuntimeError: Error(s) in loading state_dict for SpeechT5TextEncoderPrenet:
	Missing key(s) in state_dict: "encode_positions.pe". 

In [ ]:
mapping_folder = "../extracted/speecht5/mapping/"
if not os.path.exists(mapping_folder):
    os.makedirs(embeddings_folder)

with open(os.path.join(mapping_folder, 'text_prenet_state_dict.pickle'), 'wb') as handle:
    pickle.dump(speech_prenet_state_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)